In [117]:
# import the necessary packages
from imutils.perspective import four_point_transform
from imutils import contours
import imutils
import cv2
 
# define the dictionary of digit segments so we can identify
# each digit on the thermostat
DIGITS_LOOKUP = {
	(1, 1, 1, 0, 1, 1, 1): 0,
	(0, 0, 1, 0, 0, 1, 0): 1,
	(1, 0, 1, 1, 1, 1, 0): 2,
	(1, 0, 1, 1, 0, 1, 1): 3,
	(0, 1, 1, 1, 0, 1, 0): 4,
	(1, 1, 0, 1, 0, 1, 1): 5,
	(1, 1, 0, 1, 1, 1, 1): 6,
	(1, 0, 1, 0, 0, 1, 0): 7,
	(1, 1, 1, 1, 1, 1, 1): 8,
	(1, 1, 1, 1, 0, 1, 1): 9
}

In [118]:
# load the example image
image = cv2.imread("example3.jpg")
image = imutils.resize(image, height=500)
cv2.imwrite("resize.jpg", image)
cv2.imshow('R-RGB',image[:, :, 2])
cv2.imwrite('R-RGB.jpg',image[:, :, 2])

True

In [119]:
image.shape

(500, 2318, 3)

In [120]:
grayimg = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(grayimg, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 5))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
thresh = (255 - thresh)
cv2.imwrite("thresh.jpg",thresh)


True

In [121]:
# find contours in the thresholded image, then initialize the
# digit contours lists
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
digitCnts = []

# loop over the digit area candidates
for c in cnts:
    # compute the bounding box of the contour
    (x, y, w, h) = cv2.boundingRect(c)
    # if the contour is sufficiently large, it must be a digit
    if w >= 50 and (h >= 150 and h <= 350):
        digitCnts.append(c)
        cv2.imwrite(f"{x}.jpg", imutils.resize(image[ y:y+h, x:x+w], height=24, width=24))
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2) 
        print(x, y, w, h)

162 142 50 191
925 135 66 198
397 128 177 213
1109 112 157 243
1692 97 158 244
1976 88 150 283
226 64 62 251


In [122]:
len(digitCnts)

7

In [123]:
# imCopy =image.copy()

# cv2.drawContours(image, cnts, -1, (0,255,0), 3)
cv2.imwrite("count.jpg", image) 

True

In [124]:
digitCnts = contours.sort_contours(digitCnts, method="left-to-right")[0]
digits = []


len(digitCnts)

7

In [125]:
# loop over each of the digits
for c in digitCnts:
    # extract the digit ROI
    (x, y, w, h) = cv2.boundingRect(c)
    roi = thresh[y:y + h, x:x + w]

    # compute the width and height of each of the 7 segments
    # we are going to examine
    (roiH, roiW) = roi.shape
    (dW, dH) = (int(roiW * 0.25), int(roiH * 0.15))
    dHC = int(roiH * 0.05)

    # define the set of 7 segments
    segments = [
        ((0, 0), (w, dH)),	# top
        ((0, 0), (dW, h // 2)),	# top-left
        ((w - dW, 0), (w, h // 2)),	# top-right
        ((0, (h // 2) - dHC) , (w, (h // 2) + dHC)), # center
        ((0, h // 2), (dW, h)),	# bottom-left
        ((w - dW, h // 2), (w, h)),	# bottom-right
        ((0, h - dH), (w, h))	# bottom
    ]
    print([0])
    on = [0] * len(segments)
    print(on)

[0]
[0, 0, 0, 0, 0, 0, 0]
[0]
[0, 0, 0, 0, 0, 0, 0]
[0]
[0, 0, 0, 0, 0, 0, 0]
[0]
[0, 0, 0, 0, 0, 0, 0]
[0]
[0, 0, 0, 0, 0, 0, 0]
[0]
[0, 0, 0, 0, 0, 0, 0]
[0]
[0, 0, 0, 0, 0, 0, 0]


In [126]:
on

[0, 0, 0, 0, 0, 0, 0]

In [127]:
# loop over the segments
for (i, ((xA, yA), (xB, yB))) in enumerate(segments):
    # extract the segment ROI, count the total number of
    # thresholded pixels in the segment, and then compute
    # the area of the segment
    segROI = roi[yA:yB, xA:xB]
    total = cv2.countNonZero(segROI)
    area = (xB - xA) * (yB - yA)

    # if the total number of non-zero pixels is greater than
    # 50% of the area, mark the segment as "on"
    if total / float(area) > 0.5:
        on[i]= 1
    print(on)
# lookup the digit and draw it on the image
digit = DIGITS_LOOKUP[tuple(on)]
digits.append(digit)
cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 1)
cv2.putText(image, str(digit), (x - 10, y - 10),
    cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 255, 0), 2)

[1, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 0, 0]
[1, 1, 1, 1, 1, 0, 0]
[1, 1, 1, 1, 1, 0, 0]
[1, 1, 1, 1, 1, 0, 0]


KeyError: (1, 1, 1, 1, 1, 0, 0)

In [113]:
# # pre-process the image by resizing it, converting it to
# # graycale, blurring it, and computing an edge map
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# blurred = cv2.GaussianBlur(gray, (5, 5), 0)
# edged = cv2.Canny(blurred, 50, 200, 255)

In [128]:
len(digits)

0

In [129]:
cv2.imwrite("result.jpg", image) 

True

In [130]:
print(digits)

[]
